In [3]:
from gudiff_model import PDBDataSet_GraphCon
from gudiff_model.Graph_UNet import GraphUNet
from data_rigid_diffuser.diffuser import FrameDiffNoise

from se3_transformer.model.fiber import Fiber
import torch
import os
import logging
from datetime import datetime
from collections import defaultdict
import time
import tree
from se3_transformer.model.FAPE_Loss import FAPE_loss, Qs2Rs, normQ
from torch import einsum
import numpy as np
import se3_diffuse.utils as du
import util.framediff_utils as fu
from data_rigid_diffuser import rigid_utils as ru
import copy
import util.pdb_writer 
from experiment.Experiment import Experiment
import json
import argparse




/home/nwoodall/miniconda3/envs/gud/lib/python3.9/site-packages/torchdata/datapipes/__init__.py:18: UserWarning: 
################################################################################
WARNING!
The 'datapipes', 'dataloader2' modules are deprecated and will be removed in a
future torchdata release! Please see https://github.com/pytorch/data/issues/1196
to learn more and leave feedback.
################################################################################

  deprecation_warning()
/home/nwoodall/miniconda3/envs/gud/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
#mode meta

import pandas as pd

In [32]:
fn = '/mnt/h/datasets/bCov_4H/metadata.csv'
dir_path = os.path.dirname(os.path.realpath(fn))
df = pd.read_csv(fn)

dirpath = '/gdrive_Test/'

In [21]:
os.path.abspath(os.path.join(os.path.dirname(fn)))

'/mnt/h/datasets/bCov_4H'

In [24]:
dirpath 

'/gdrive_Test/'

In [28]:
fn1 = df['processed_path'].iloc[0]

In [29]:
fn1

'/mnt/h/datasets/bCov_4H/00/0001581c72611c299a7297cf4c1d9851_0001.pkl'

In [31]:
os.path.basename(os.path.dirname(fn1))

'00'

In [35]:
df['processed_path'] = df['processed_path'].apply(lambda x: os.path.join(  dirpath,  os.path.basename((os.path.dirname(x))),  os.path.basename(x)) )

In [11]:
os.path.basename(x)

In [36]:
df['processed_path'].iloc[0]

'/gdrive_Test/00/0001581c72611c299a7297cf4c1d9851_0001.pkl'

In [15]:
df

,pdb_name,processed_path,raw_path,oligomeric_count,oligomeric_detail,resolution,structure_method,num_chains,quaternary_category,seq_len,modeled_seq_len,coil_percent,helix_percent,strand_percent,radius_gyration
0,0001581c72611c299a7297cf4c1d9851_0001,/mnt/h/datasets/bCov_4H/00/0001581c72611c299a7...,/mnt/c/Users/nwood/OneDrive/Desktop/hTest/BCov...,NaN,NaN,0,generated,1,homomer,65,65,0.138462,0.861538,0.0,1.111881
1,0002182ba9e87d7b0d6b7d16b7001923_0001,/mnt/h/datasets/bCov_4H/00/0002182ba9e87d7b0d6...,/mnt/c/Users/nwood/OneDrive/Desktop/hTest/BCov...,NaN,NaN,0,generated,1,homomer,65,65,0.200000,0.800000,0.0,1.133914
2,00052a785cf4cffaa60b15d2ccc78f9d_0001_1,/mnt/h/datasets/bCov_4H/00/00052a785cf4cffaa60...,/mnt/c/Users/nwood/OneDrive/Desktop/hTest/BCov...,NaN,NaN,0,generated,1,homomer,65,65,0.184615,0.815385,0.0,1.111930
3,000701cc0c20e04627e1e44f84d5a6b6_0001_1,/mnt/h/datasets/bCov_4H/00/000701cc0c20e04627e...,/mnt/c/Users/nwood/OneDrive/Desktop/hTest/BCov...,NaN,NaN,0,generated,1,homomer,65,65,0.200000,0.800000,0.0,1.111956
4,000877380acb95e21726d9df052ec962_0001,/mnt/h/datasets/bCov_4H/00/000877380acb95e2172...,/mnt/c/Users/nwood/OneDrive/Desktop/hTest/BCov...,NaN,NaN,0,generated,1,homomer,65,65,0.200000,0.800000,0.0,1.104479
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27889,fff532a1f3bd52993b00d58dc306e14c_0001_1,/mnt/h/datasets/bCov_4H/ff/fff532a1f3bd52993b0...,/mnt/c/Users/nwood/OneDrive/Desktop/hTest/BCov...,NaN,NaN,0,generated,1,homomer,65,65,0.184615,0.815385,0.0,1.090073
27890,fff8daf3a52878a3bf78d4aa4fd68612_0001_1,/mnt/h/datasets/bCov_4H/ff/fff8daf3a52878a3bf7...,/mnt/c/Users/nwood/OneDrive/Desktop/hTest/BCov...,NaN,NaN,0,generated,1,homomer,65,65,0.169231,0.830769,0.0,1.098638
27891,fffa17b132597274c1fd4fd217829317_0001_1,/mnt/h/datasets/bCov_4H/ff/fffa17b132597274c1f...,/mnt/c/Users/nwood/OneDrive/Desktop/hTest/BCov...,NaN,NaN,0,generated,1,homomer,65,65,0.200000,0.800000,0.0,1.111163
27892,fffaaf5372bf5d4f67462faa60fe4466_0001,/mnt/h/datasets/bCov_4H/ff/fffaaf5372bf5d4f674...,/mnt/c/Users/nwood/OneDrive/Desktop/hTest/BCov...,NaN,NaN,0,generated,1,homomer,65,65,0.184615,0.815385,0.0,1.099117


In [13]:
df['processed_path']

0        /gdrive_Test/0001581c72611c299a7297cf4c1d9851_...
1        /gdrive_Test/0002182ba9e87d7b0d6b7d16b7001923_...
2        /gdrive_Test/00052a785cf4cffaa60b15d2ccc78f9d_...
3        /gdrive_Test/000701cc0c20e04627e1e44f84d5a6b6_...
4        /gdrive_Test/000877380acb95e21726d9df052ec962_...
                               ...                        
27889    /gdrive_Test/fff532a1f3bd52993b00d58dc306e14c_...
27890    /gdrive_Test/fff8daf3a52878a3bf78d4aa4fd68612_...
27891    /gdrive_Test/fffa17b132597274c1fd4fd217829317_...
27892    /gdrive_Test/fffaaf5372bf5d4f67462faa60fe4466_...
27893    /gdrive_Test/fffab47f11ab9cbb43a0143b90b3e454_...
Name: processed_path, Length: 27894, dtype: object

In [ ]:
with open('configs/base_gun.json','r') as f:
    conf = json.load(f)



if __name__ == '__main__':
    parser = argparse.ArgumentParser('Train Graph U-Net')
    parser.add_argument('name', help='name of run',type=str)

    group = parser.add_mutually_exclusive_group(required=True)
    group.add_argument("-c", '--config_path',  help='path to config file in json format',type=str)
    group.add_argument("-k", '--checkpoint_file', help='previous checkpoint to load',type=str)

    args = parser.parse_args()

    if args.checkpoint_file is not None:

        ckpt_model=torch.load(args.checkpoint_file)['model']
        ckpt_opt = torch.load(args.checkpoint_file)['optimizer']
        conf = torch.load(args.checkpoint_file)['conf']

        exp = Experiment(conf,name=args.name, ckpt_model=ckpt_model, ckpt_opt=ckpt_opt)

    else:
        with open(args.config_path, 'r') as f:
            conf = json.load(f)
        exp = Experiment(conf, name=args.name, ckpt_model=None, ckpt_opt=None)

    tl, vl = exp.create_dataset() #load dataset from 'meta_data_path.csv'
    exp.start_training()